In [1]:
#df =pd.read_csv("International_Report_Passengers.csv",index_col="data_dte",parse_dates=["data_dte"])
#df1 = pd.DataFrame(df, columns = ['fg_wac', 'Total'])
#df1
#dataset.dropna(inplace=True)

In [2]:
#del(df['usg_apt_id'])
#del(df['Scheduled'])
#del(df['Charter'])
#del(df['usg_apt'])
#del(df['fg_apt'])
#del(df['year'])
#del(df['month'])
#Nos quedamos con las variables más relevantes
#df = pd.DataFrame(df, columns = ['fg_wac', 'Total'])

# Preprocesamiento

## Importacion de librerias

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,LSTM

## Carga del dataset y seleccion de columnas

In [4]:
#df =pd.read_csv("International_Report_Passengers.csv",parse_dates=["data_dte"])
df = pd.read_csv("/kaggle/input/us-international-air-traffic-data/International_Report_Passengers.csv",parse_dates=["data_dte"])


#df = pd.DataFrame(df, columns = ['data_dte','fg_wac', 'Total'])
#datasetob1=df
df

,data_dte,Year,Month,usg_apt_id,usg_apt,usg_wac,fg_apt_id,fg_apt,fg_wac,airlineid,carrier,carriergroup,type,Scheduled,Charter,Total
0,2014-05-01,2014,5,14492,RDU,36,11032,CUN,148,19534,AM,0,Passengers,0,315,315
1,2007-06-01,2007,6,13204,MCO,33,16085,YHZ,951,20364,C6,0,Passengers,0,683,683
2,2005-12-01,2005,12,11433,DTW,43,10411,AUA,277,20344,RD,1,Passengers,0,1010,1010
3,2003-04-01,2003,4,13487,MSP,63,16304,ZIH,148,20204,MG,1,Passengers,0,508,508
4,2005-12-01,2005,12,12016,GUM,5,11138,CRK,766,20312,TZ,1,Passengers,0,76,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680980,2015-10-01,2015,10,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,130217,0,130217
680981,2014-09-01,2014,9,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,132052,0,132052
680982,2014-10-01,2014,10,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,132822,0,132822
680983,2014-08-01,2014,8,12478,JFK,22,12972,LHR,493,19540,BA,0,Passengers,134263,0,134263


### Descripcion de variables

In [5]:
df.dtypes

data_dte        datetime64[ns]
Year                     int64
Month                    int64
usg_apt_id               int64
usg_apt                 object
usg_wac                  int64
fg_apt_id                int64
fg_apt                  object
fg_wac                   int64
airlineid                int64
carrier                 object
carriergroup             int64
type                    object
Scheduled                int64
Charter                  int64
Total                    int64
dtype: object

### Identificación de valores faltantes

In [6]:
#Pasamos caracteres raros a nan para poder identificarlos
df[df == '?'] = np.nan

#Hacemos conteo de los valores nulos
print(df.isnull().sum())

data_dte           0
Year               0
Month              0
usg_apt_id         0
usg_apt            0
usg_wac            0
fg_apt_id          0
fg_apt             0
fg_wac             0
airlineid          0
carrier         2797
carriergroup       0
type               0
Scheduled          0
Charter            0
Total              0
dtype: int64


### Tratamiento de valores faltantes

In [7]:
df=df.dropna(axis = 1)
#Pasamos caracteres raros a nan para poder identificarlos
df[df == '?'] = np.nan

#Hacemos conteo de los valores nulos
print(df.isnull().sum())

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3718: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


data_dte        0
Year            0
Month           0
usg_apt_id      0
usg_apt         0
usg_wac         0
fg_apt_id       0
fg_apt          0
fg_wac          0
airlineid       0
carriergroup    0
type            0
Scheduled       0
Charter         0
Total           0
dtype: int64


### Analizamos caracteristicas

In [8]:
input_data=df[df.columns[:-1]]
input_data.skew() 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


Year            -0.192840
Month            0.058169
usg_apt_id      25.097443
usg_wac          0.489536
fg_apt_id        0.005399
fg_wac           0.465334
airlineid        1.446007
carriergroup    -0.250882
Scheduled        2.738361
Charter         11.635794
dtype: float64

In [9]:
#input_data.plot(kind="density",subplots=True,layout=(4,4),sharex=False,sharey=False)

<h1>Jose, pongo aquí lo que he hecho para deputar el dataset.</h1>

In [10]:
import requests
from bs4 import BeautifulSoup as bs

# Cargo el diccionario de paises

url = 'https://github.com/stefangabos/world_countries/blob/master/data/countries/cs/countries.csv'
content = requests.get(url).content
soup = bs(content, "html.parser")
datos = soup.find_all('tr', attrs={'class': 'js-file-line'})

paises_alfa = {}

for row in datos:
    cell = [i.text for i in row.find_all('td')]
    if len(cell) > 1:
        paises_alfa[cell[3].upper()] = cell[4]
#paises_alfa

In [11]:
# Diccionario de países por código numérico

url = 'https://github.com/tasneem/greenhouse-emitters/blob/master/data/lookup-tables/L_WORLD_AREA_CODES.csv'
content = requests.get(url).content
soup = bs(content, "html.parser")
datos = soup.find_all('tr', attrs={'class': 'js-file-line'})

paises_num = {}

for row in datos:
    cell = [i.text for i in row.find_all('td')]
    if len(cell) > 1:
        paises_num[cell[1].upper()] = cell[2]
        
#paises_num

In [12]:
# Diccionario de aeropuertos. Columna fg_apt_id

url = 'https://github.com/tasneem/greenhouse-emitters/blob/master/data/lookup-tables/L_AIRPORT_ID.csv'
content = requests.get(url).content
soup = bs(content, "html.parser")
datos = soup.find_all('tr', attrs={'class': 'js-file-line'})

aeropuertos = {}

for row in datos:
    cell = [i.text for i in row.find_all('td')]
    if len(cell) > 1:
        aeropuertos[int(cell[1])] = cell[2]
        
#aeropuertos

In [13]:
# Diccionario de compañías aéreas. Código alfanumérico 3 dig. Columna carrier

url = 'https://github.com/tasneem/greenhouse-emitters/blob/master/data/lookup-tables/L_UNIQUE_CARRIERS.csv'
content = requests.get(url).content
soup = bs(content, "html.parser")
datos = soup.find_all('tr', attrs={'class': 'js-file-line'})

dic_carrier = {}

for row in datos:
    cell = [i.text for i in row.find_all('td')]
    if len(cell) > 1:
        dic_carrier[cell[1].upper()] = cell[2]
        
#dic_carrier

In [14]:
# Diccionario de compañía aérea. Código numérico de 5 dig. Columna airlineid

import csv

with open("/kaggle/input/airline-codes-mapping/L_AIRLINE_ID.csv") as file:
    reader = csv.reader(file)
    next(reader) # Descartamos cabecera
    dic_airlineid = dict(reader)
#dic_airlineid

In [15]:
datos = pd.read_csv('../input/us-international-air-traffic-data/International_Report_Passengers.csv')
# Eliminamos las columnas usg_apt y fg_apt puesto que contienen el mismo dato que las columnas usg_apt_id y fg_apt_id respectivamente
# Eliminamos las columnas que nos indican de donde parten los viajeros. Lo que nos importa es el destino.
# Eliminamos la columna que nos indica el número de pasageros en vuelos charter. Nos interesa el número total.

datos.drop(['type','Scheduled','usg_apt','usg_apt_id','usg_wac','fg_apt','carriergroup','Charter'], axis=1, inplace=True)
datos.head()

,data_dte,Year,Month,fg_apt_id,fg_wac,airlineid,carrier,Total
0,05/01/2014,2014,5,11032,148,19534,AM,315
1,06/01/2007,2007,6,16085,951,20364,C6,683
2,12/01/2005,2005,12,10411,277,20344,RD,1010
3,04/01/2003,2003,4,16304,148,20204,MG,508
4,12/01/2005,2005,12,11138,766,20312,TZ,76


In [16]:
# Agrupo y sumo los datos por año, mes y destino

datos_agrupados = datos.groupby(['Year','Month','fg_apt_id']).agg(
                                  {'Total': 'sum',
                                  }).reset_index()

nom_aeropuertos = [aeropuertos.get(x) for x in datos_agrupados['fg_apt_id']]
datos_agrupados['aeropuerto'] = nom_aeropuertos

datos_agrupados

,Year,Month,fg_apt_id,Total,aeropuerto
0,1990,1,10148,58212,"Acapulco, Mexico: General Juan N Alvarez Inter..."
1,1990,1,10174,1309,"San Andres Island, Colombia: Gustavo Rojas Pin..."
2,1990,1,10205,2543,"Malaga, Spain: Malaga Costa del Sol"
3,1990,1,10244,64280,"Auckland, New Zealand: Auckland International"
4,1990,1,10287,967,"Amman, Jordan: Queen Alia International"
...,...,...,...,...,...
94334,2020,3,16825,36,None
94335,2020,3,16835,1,None
94336,2020,3,16845,28,None
94337,2020,3,16858,96583,None


<h1>Hasta aquí lo de Pablo</h1>

### Eliminamos columnas que no son relevantes

In [17]:
df = pd.DataFrame(df, columns = ['data_dte','fg_wac', 'Total'])
#datasetob1=df

## División por pais

### Carga de codigos WAC para poder saber qué codigo es cada pais

In [18]:
destinos =pd.read_csv("/kaggle/input/worldareacodes/L_WORLD_AREA_CODES.csv")
destinos

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/worldareacodes/L_WORLD_AREA_CODES.csv'

### Estudio de la distribución para seleccionar destinos más relevantes

In [ ]:
obejtivos = df.groupby('fg_wac').size()
print(obejtivos.sort_values().tail())
print(df["fg_wac"].hist())

In [ ]:
obejtivo1 = destinos[destinos['Code'] == 204]
obejtivo2 = destinos[destinos['Code'] == 736]
obejtivo3 = destinos[destinos['Code'] == 493]
obejtivo4 = destinos[destinos['Code'] == 936]
obejtivo5 = destinos[destinos['Code'] == 148]
print(obejtivo1)
print(obejtivo2)
print(obejtivo3)
print(obejtivo4)
print(obejtivo5)

### Creamos dataset filtrado con  uno de los paises más recurridos 

In [ ]:
#datasetob1 = df[df['fg_wac'] == 204]
#datasetob2 = df[df['fg_wac'] == 736]
#datasetob3 = df[df['fg_wac'] == 493]
#datasetob4 = df[df['fg_wac'] == 936]
#datasetob5 = df[df['fg_wac'] == 148]
datasetob1 = df[df['fg_wac'] == 148]

### Agrupado y sumado del total de pasageros por año

In [ ]:
#datasetob1 = df[df['fg_wac'] == 204]
datasetob1 = pd.DataFrame(datasetob1, columns = ['data_dte', 'Total'])
datasetob1=datasetob1.groupby(by=['data_dte']).sum().groupby(level=[0]).cumsum()
#datasetob1.set_index('Year')
datasetob1
#datasetob1 = pd.DataFrame(datasetob1, columns = ['Total'])
#dataset.dropna(inplace=True)

# Proceso de Predicción

In [ ]:
#ahora solo necesitaremos el total y la fecha para la prediccion
#datasetob1 = pd.DataFrame(datasetob1, columns = ['Total'])
train = pd.DataFrame(datasetob1.Total[:"2018"])
test = pd.DataFrame(datasetob1.Total["2019":])
train.tail(100)

In [ ]:
minmaxSc=MinMaxScaler()
sc_train= minmaxSc.fit_transform(train)
windows = 5
X_train = []
Y_train = []
#print(windows)
#rint(sc_train.size)
for i in range(windows,sc_train.size):
    X_train.append(sc_train[i-windows:i,0])
    Y_train.append(sc_train[i,0])
print(X_train)

In [ ]:
X_train,Y_train = np.array(X_train),np.array(Y_train)
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
modelo=Sequential()
modelo.add(LSTM(units=600, input_shape=(X_train.shape[1],1)))
modelo.add(Dense(1))
modelo.compile(optimizer="adam",loss="mse")
modelo.fit(X_train,Y_train,epochs=10,batch_size=64)

In [ ]:
sc_test = minmaxSc.transform(test)
X_test = []
for i in range(windows,sc_test.size):
    X_test.append(sc_test[i-windows:i,0])

In [ ]:
X_test=np.array(X_test)
X_test= np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
pred=modelo.predict(X_test)
pred=minmaxSc.inverse_transform(pred)

In [ ]:
test["pred"]=np.concatenate(([np.nan]*windows,pred),axis=None)
plt.plot(test["Total"],color="darkcyan")
plt.plot(test["pred"],color="darkslategrey")
#plt.ylim(1.1*np.min(pred)/2, 1.1 *np.max(pred))
plt.show()